# Template 

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [1]:
!pip3 install -U random-forest-mc

In [2]:
%load_ext watermark

In [16]:
from random_forest_mc.model import RandomForestMC
from random_forest_mc.utils import load_file_json
from collections import Counter
import pandas as pd
import numpy as np

In [10]:
from tqdm import tqdm

# from glob import glob

# import matplotlib.pyplot as plt
# %matplotlib inline
# from matplotlib import rcParams
# from cycler import cycler

# rcParams['figure.figsize'] = 12, 8 # 18, 5
# rcParams['axes.spines.top'] = False
# rcParams['axes.spines.right'] = False
# rcParams['axes.grid'] = True
# rcParams['axes.prop_cycle'] = cycler(color=['#365977'])
# rcParams['lines.linewidth'] = 2.5

# import seaborn as sns
# sns.set_theme()

# pd.set_option("max_columns", None)
# pd.set_option("max_rows", None)
# pd.set_option('display.max_colwidth', None)

# from IPython.display import Markdown, display
# def md(arg):
#     display(Markdown(arg))

# from pandas_profiling import ProfileReport
# #report = ProfileReport(#DataFrame here#, minimal=True)
# #report.to

# import pyarrow.parquet as pq
# #df = pq.ParquetDataset(path_to_folder_with_parquets, filesystem=None).read_pandas().to_pandas()

# import json
# def open_file_json(path,mode='r',var=None):
#     if mode == 'w':
#         with open(path,'w') as f:
#             json.dump(var, f)
#     if mode == 'r':
#         with open(path,'r') as f:
#             return json.load(f)

# import functools
# import operator
# def flat(a):
#     return functools.reduce(operator.iconcat, a, [])

# import json
# from glob import glob
# from typing import NewType


# DictsPathType = NewType("DictsPath", str)


# def open_file_json(path):
#     with open(path, "r") as f:
#         return json.load(f)

# class LoadDicts:
#     def __init__(self, dict_path: DictsPathType = "./data"):
#         Dicts_glob = glob(f"{dict_path}/*.json")
#         self.List = []
#         self.Dict = {}
#         for path_json in Dicts_glob:
#             name = path_json.split("/")[-1].replace(".json", "")
#             self.List.append(name)
#             self.Dict[name] = open_file_json(path_json)
#             setattr(self, name, self.Dict[name])


In [11]:
# Run this cell before close.
%watermark -d --iversion -b -r -g -m -v
!cat /proc/cpuinfo |grep 'model name'|head -n 1 |sed -e 's/model\ name/CPU/'
!free -h |cut -d'i' -f1  |grep -v total

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.27.0

Compiler    : GCC 10.2.1 20210110
OS          : Linux
Release     : 5.11.0-7633-generic
Machine     : x86_64
Processor   : 
CPU cores   : 4
Architecture: 64bit

Git hash: 1455288e7e5a975c6dcb21602b1eaf2480c2aeb5

Git repo: https://github.com/ysraell/random-forest-mc.git

Git branch: main

CPU	: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz
Mem:            15G
Swap:          4.0G


In [6]:
modeldict = load_file_json('/work/data/cls_rfmc_v030.json')
modeldict.keys()

dict_keys(['target_col', 'batch_train_pclass', 'batch_val_pclass', '_N', 'min_feature', 'max_feature', 'th_start', 'delta_th', 'max_discard_trees', 'n_trees', 'class_vals', 'Forest', 'survived_scores', 'version'])

In [7]:
cls = RandomForestMC()
for att in ['numeric_cols','feature_cols','type_of_cols','target_col','class_vals']:
    modeldict[att] = None
modeldict['version'] = '0.3.0'
cls.dict2model(modeldict)

In [8]:
len(cls.Forest)

128

In [20]:
cls.target_col = 'Class'

In [21]:
df = pd.read_csv('/work/data/creditcard_trans_int.csv')
target_col = 'Class'
ds_cols = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
df = df[ds_cols+[target_col]].dropna()
cls.process_dataset(df)

# Count

In [115]:
"\'V1\'" in str(cls.Forest[0])

True

In [116]:
feat_counts = {feat: sum([f"'{feat}'" in str(Tree) for Tree in cls.Forest])/len(cls.Forest) for feat in cls.feature_cols}
sorted(feat_counts.items(), key=lambda x: x[1], reverse=True)

[('V14', 0.8046875),
 ('V4', 0.75),
 ('V16', 0.4375),
 ('V12', 0.4296875),
 ('V3', 0.3984375),
 ('V10', 0.3984375),
 ('V17', 0.375),
 ('V5', 0.34375),
 ('V11', 0.34375),
 ('Amount', 0.328125),
 ('V1', 0.3203125),
 ('V2', 0.3046875),
 ('V7', 0.2890625),
 ('V19', 0.2890625),
 ('V8', 0.28125),
 ('V18', 0.28125),
 ('V15', 0.2578125),
 ('V21', 0.2578125),
 ('V24', 0.234375),
 ('V9', 0.2265625),
 ('V23', 0.2265625),
 ('V28', 0.2265625),
 ('V20', 0.203125),
 ('V22', 0.203125),
 ('V6', 0.1953125),
 ('V13', 0.1796875),
 ('V26', 0.1640625),
 ('V27', 0.1484375),
 ('V25', 0.125)]

In [129]:
sum_scores = sum(cls.survived_scores)
feat_counts = {feat: np.mean(list(filter(lambda num: num != 0,[(f"'{feat}'" in str(Tree))*score for Tree,score in zip(cls.Forest,cls.survived_scores)]))) for feat in cls.feature_cols}
sorted(feat_counts.items(), key=lambda x: x[1], reverse=True)

[('V22', 0.9237355110642783),
 ('V8', 0.9237062404870624),
 ('V20', 0.9234720758693361),
 ('V21', 0.9232046492320464),
 ('V28', 0.9222957014643363),
 ('V16', 0.922272504892368),
 ('V5', 0.9220890410958907),
 ('V11', 0.9220890410958907),
 ('V4', 0.9220890410958905),
 ('V3', 0.9219715283373626),
 ('V19', 0.9217882265827473),
 ('V25', 0.9214469178082192),
 ('V14', 0.9211663785077804),
 ('V13', 0.9209350804050026),
 ('Amount', 0.9208251793868234),
 ('V7', 0.9207700851536469),
 ('V18', 0.9207572298325724),
 ('V10', 0.9206956755304861),
 ('V1', 0.9206481790845308),
 ('V6', 0.9205479452054794),
 ('V12', 0.9203611457036116),
 ('V17', 0.9202340182648401),
 ('V9', 0.919933868682097),
 ('V27', 0.9197909156452774),
 ('V15', 0.9197799916977998),
 ('V2', 0.9193010186160874),
 ('V26', 0.9192759295499021),
 ('V23', 0.9188710439300894),
 ('V24', 0.9187214611872143)]

In [26]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier()
forest.fit(df[ds_cols], df[target_col])

RandomForestClassifier()

In [35]:
feat_counts_baseline = {feat: score for feat,score in zip(ds_cols, forest.feature_importances_)}

In [36]:
sorted(feat_counts_baseline.items(), key=lambda x: x[1], reverse=True)

[('V12', 0.145750137242988),
 ('V17', 0.13073994696652233),
 ('V14', 0.10869943505722834),
 ('V10', 0.10659552465814812),
 ('V11', 0.07494282109076995),
 ('V16', 0.07007434988401999),
 ('V9', 0.03723455894197347),
 ('V18', 0.031377649659517226),
 ('V7', 0.02974542566681773),
 ('V4', 0.02442541464406295),
 ('V26', 0.020258285882117703),
 ('V21', 0.01801605889498627),
 ('V1', 0.01675503823468746),
 ('V6', 0.016007484083145247),
 ('V3', 0.014916958829971082),
 ('V20', 0.01349498078472359),
 ('V27', 0.012218137804112314),
 ('V8', 0.011852712616607844),
 ('V19', 0.011730061830087143),
 ('V5', 0.011602808187383826),
 ('Amount', 0.01133494532179039),
 ('V2', 0.011281574126221659),
 ('V28', 0.011259359502862502),
 ('V13', 0.01094957985420983),
 ('V22', 0.010747722550576653),
 ('V25', 0.010685071237540667),
 ('V15', 0.010582035128143272),
 ('V24', 0.009877699910013103),
 ('V23', 0.006844221408771363)]

In [126]:
ForestFake = [str(cls.sampleFeats()) for _ in range(1024*10)]
feat_counts = {feat: sum([f"'{feat}'" in str(Tree) for Tree in ForestFake])/len(ForestFake) for feat in cls.feature_cols}
sorted(feat_counts.items(), key=lambda x: x[1], reverse=True)

[('V6', 0.7501953125),
 ('V4', 0.74755859375),
 ('V28', 0.747265625),
 ('V11', 0.74677734375),
 ('V5', 0.746484375),
 ('V7', 0.74521484375),
 ('V12', 0.74521484375),
 ('V15', 0.74482421875),
 ('V16', 0.7447265625),
 ('V2', 0.74462890625),
 ('V3', 0.744140625),
 ('V13', 0.7431640625),
 ('Amount', 0.7431640625),
 ('V18', 0.74267578125),
 ('V19', 0.7423828125),
 ('V20', 0.7423828125),
 ('V17', 0.7419921875),
 ('V8', 0.7416015625),
 ('V14', 0.74130859375),
 ('V26', 0.7408203125),
 ('V9', 0.7404296875),
 ('V10', 0.74013671875),
 ('V25', 0.74013671875),
 ('V22', 0.739453125),
 ('V23', 0.739453125),
 ('V1', 0.738671875),
 ('V24', 0.73740234375),
 ('V27', 0.7353515625),
 ('V21', 0.73486328125)]